In [30]:
# Import required libraries for database connection and data handling 
from sqlalchemy import create_engine
import pandas as pd

import os  # Import the os module to interact with the operating system
from dotenv import load_dotenv  # Import the load_dotenv function to load environment variables from a .env file

import openai  # Import the OpenAI library to interact with the OpenAI API


In [31]:


# Set up the connection string for SQL Server using SQLAlchemy
connection_string = (
    "mssql+pyodbc://"
    "DESKTOP-SNEPGI0/"  # Server name
    "AdventureWorks2019?"  # Database name
    "driver=ODBC+Driver+17+for+SQL+Server&"  # ODBC driver
    "trusted_connection=yes"  # Use Windows authentication
)

# Create a SQLAlchemy engine for managing the database connection
engine = create_engine(connection_string)

# Define a function to execute SQL queries
def execute_query(query, engine):
    """
    Executes an SQL query and returns the result as a DataFrame.
    """
    try:
        result_df = pd.read_sql(query, engine)
        return result_df
    except Exception as e:
        print(f"Error executing query: {e}")
        return None

# Test the function with a sample query
query = """
SELECT SalesOrderID, SubTotal, TaxAmt, Freight, TotalDue
FROM Sales.SalesOrderHeader
"""

#result = execute_query(query, engine)
df = execute_query(query, engine)
print(df.head())
#if result is not None:
    #print("Query Result:\n", result.head())


   SalesOrderID    SubTotal     TaxAmt   Freight    TotalDue
0         43659  20565.6206  1971.5149  616.0984  23153.2339
1         43660   1294.2529   124.2483   38.8276   1457.3288
2         43661  32726.4786  3153.7696  985.5530  36865.8012
3         43662  28832.5289  2775.1646  867.2389  32474.9324
4         43663    419.4589    40.2681   12.5838    472.3108


In [32]:
# # Load environment variables from a .env file
# load_dotenv('.env')

# # Retrieve the OpenAI API key from the environment variables
# openai_api_key = os.environ["OPENAI_API_KEY"]
# # Set the OpenAI API key
# openai.api_key = openai_api_key

In [33]:
schema_query = """
SELECT TABLE_SCHEMA,TABLE_NAME, COLUMN_NAME, DATA_TYPE
FROM INFORMATION_SCHEMA.COLUMNS
WHERE TABLE_CATALOG = 'AdventureWorks2019';
"""
schema_result = execute_query(schema_query, engine)

if schema_result is not None:
    # Display the schema information
    schema_info = "\n".join(
        #f"{row['TABLE_NAME']}: {row['COLUMN_NAME']} ({row['DATA_TYPE']})"
        f"{row['TABLE_SCHEMA']}.{row['TABLE_NAME']}.{row['COLUMN_NAME']} ({row['DATA_TYPE']})"
                for _, row in schema_result.iterrows() 
    )
    print("Schema Information:")
    print(schema_info)
    


Schema Information:
Sales.SalesTaxRate.SalesTaxRateID (int)
Sales.SalesTaxRate.StateProvinceID (int)
Sales.SalesTaxRate.TaxType (tinyint)
Sales.SalesTaxRate.TaxRate (smallmoney)
Sales.SalesTaxRate.Name (nvarchar)
Sales.SalesTaxRate.rowguid (uniqueidentifier)
Sales.SalesTaxRate.ModifiedDate (datetime)
Sales.PersonCreditCard.BusinessEntityID (int)
Sales.PersonCreditCard.CreditCardID (int)
Sales.PersonCreditCard.ModifiedDate (datetime)
Person.vAdditionalContactInfo.BusinessEntityID (int)
Person.vAdditionalContactInfo.FirstName (nvarchar)
Person.vAdditionalContactInfo.MiddleName (nvarchar)
Person.vAdditionalContactInfo.LastName (nvarchar)
Person.vAdditionalContactInfo.TelephoneNumber (nvarchar)
Person.vAdditionalContactInfo.TelephoneSpecialInstructions (nvarchar)
Person.vAdditionalContactInfo.Street (nvarchar)
Person.vAdditionalContactInfo.City (nvarchar)
Person.vAdditionalContactInfo.StateProvince (nvarchar)
Person.vAdditionalContactInfo.PostalCode (nvarchar)
Person.vAdditionalContactInfo

In [34]:
# Check for null values in the DataFrame and print the count of nulls for each column
if df is not None:
    print("Count of null values in each column:")
    print(df.isnull().sum())

Count of null values in each column:
SalesOrderID    0
SubTotal        0
TaxAmt          0
Freight         0
TotalDue        0
dtype: int64


In [35]:
# Identify rows with invalid values (negative SubTotal, TaxAmt, or Freight)
invalid_values = df[(df['SubTotal'] < 0) | (df['TaxAmt'] < 0) | (df['Freight'] < 0)]
print("Rows with invalid values:")
print(invalid_values)

Rows with invalid values:
Empty DataFrame
Columns: [SalesOrderID, SubTotal, TaxAmt, Freight, TotalDue]
Index: []


In [36]:
def split_schema_into_chunks(schema_info, max_tokens=1000):
    chunks = []
    current_chunk = []
    current_length = 0

    for line in schema_info.split("\n"):
        line_length = len(line.split())
        if current_length + line_length > max_tokens:
            chunks.append("\n".join(current_chunk))
            current_chunk = []
            current_length = 0

        current_chunk.append(line)
        current_length += line_length

    if current_chunk:
        chunks.append("\n".join(current_chunk))
    return chunks



In [37]:
# Load environment variables from a .env file
load_dotenv('.env')

# Retrieve the OpenAI API key from the environment variables
openai_api_key = os.environ["OPENAI_API_KEY"]
# Set the OpenAI API key
openai.api_key = openai_api_key

In [38]:
# def natural_to_sql(question, database_name="AdventureWorks2019"):
#     """
#     Convert a natural language question about the AdventureWorks database to SQL using GPT-4.
    
#     Parameters:
#     question (str): The natural language question to convert to SQL.
#     database_name (str): The name of the database to query. Default is "AdventureWorks2019".
    
#     Returns:
#     str: The SQL query generated by GPT-4.
#     """
#     # Create a chat completion request to the OpenAI API using the GPT-4 model
#     # client = openai.Client()
#     response = openai.chat.completions.create(
#         model="gpt-4",
#         messages=[
#             {
#                 "role": "system",
#                 "content": (
#                     f"You are a SQL assistant for the {database_name} database. "
#                     f"Here is the schema of the database:\n{schema_info}\n"
#                     "Your task is to generate only the SQL query for the following request. Do not include any text, explanation, or additional characters."
#                 )
#             },
#             {"role": "user", "content": f"Generate only a single SQL query for this question whitout any explanation: {question}"}
#         ]
#     )
#     # Return the generated SQL query from the response
#     return response.choices[0].message.content
#     #print(response.choices[0].message.content)


In [39]:
def natural_to_sql(question, schema_chunks, database_name="AdventureWorks2019"):
    generated_queries = []
    client = openai.Client()
    for idx, chunk in enumerate(schema_chunks):
        try:
            response = openai.chat.completions.create(
                model="gpt-4",
                messages=[
                    {
                        "role": "system",
                        "content": (
                            f"You are a SQL assistant for the {database_name} database. "
                            f"Here is part of the database schema:\n{chunk}\n"
                            "Your task is to generate only the SQL query for the following request. Do not include any text, explanation, or additional characters."
                        )
                    },
                    {"role": "user", "content": question}
                ]
            )
            query = response.choices[0].message.content #response["choices"][0]["message"]["content"]
            generated_queries.append(query)
        except Exception as e:
            print(f"Error during query generation: {e}")

    final_query = "\n".join(generated_queries)
    return final_query


In [40]:
# Example usage:
question ="What was the sales volume of the best product in each region in year 2011?"
#print("Question:", question)
sql_query = natural_to_sql(question, schema_chunks = split_schema_into_chunks(schema_info))
print(sql_query)

SELECT 
    R.Name AS RegionName, 
    P.Name AS ProductName, 
    MAX(OD.OrderQty) AS SalesVolume
FROM 
    Sales.SalesOrderDetail AS OD
JOIN 
    Sales.SalesOrderHeader AS OH 
    ON OD.SalesOrderID = OH.SalesOrderID
JOIN 
    Sales.SalesTerritory AS R 
    ON OH.TerritoryID = R.TerritoryID
JOIN 
    Production.Product AS P 
    ON OD.ProductID = P.ProductID
WHERE 
    YEAR(OH.OrderDate) = 2011
GROUP BY 
    R.Name, 
    P.Name;
SELECT 
    S.TerritoryID, 
    sd.ProductID,
    SUM(sd.OrderQty) as SalesVolume
FROM 
    Sales.SalesOrderHeader AS S 
JOIN 
    Sales.SalesOrderDetail AS sd 
ON 
    S.SalesOrderID = sd.SalesOrderID
WHERE 
    YEAR(S.OrderDate) = 2011
GROUP BY 
    S.TerritoryID, 
    sd.ProductID
ORDER BY 
    SalesVolume DESC;


In [41]:
def execute_query(query, engine):
    """
    Executes an SQL query and returns the result as a DataFrame.
    """
    try:
        result_df = pd.read_sql(query, engine)
        return result_df
    except Exception as e:
        print(f"Error executing query: {e}")
        return None


In [42]:
result = execute_query(sql_query, engine)
if result is not None:
        print("Query Result:\n", result)

Query Result:
          RegionName                     ProductName  SalesVolume
0           Germany              Road-650 Black, 52            1
1           Germany              Road-650 Black, 62            1
2    United Kingdom                Road-150 Red, 44            1
3         Northwest           Sport-100 Helmet, Red           10
4            Canada         LL Road Frame - Red, 52            1
..              ...                             ...          ...
416       Northwest                Road-450 Red, 58            5
417       Southwest         LL Road Frame - Red, 62            5
418       Southwest   HL Mountain Frame - Black, 48            3
419       Southwest  HL Mountain Frame - Silver, 48            4
420       Southwest                Road-150 Red, 56            5

[421 rows x 3 columns]
